In [2]:
import os
import pyodbc
import pandas as pd
import numpy as np

In [3]:
# 建立连接并列出表
def get_tables(conn_str):
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    try:
        # 获取所有表名
        tables = cursor.tables(tableType='TABLE')
        tables_name = [table.table_name for table in tables]
        return tables_name
    finally:
        cursor.close()
        conn.close()

# 读取表数据，返回一个pandas的DataFrame
def get_table_data(conn_str,table_name):
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    try:
        cursor.execute(f"SELECT * FROM {table_name}")
        # 获取列名
        columns = [column[0] for column in cursor.description]
        # 获取数据并创建DataFrame,指定列名
        data = pd.DataFrame.from_records(cursor.fetchall(), columns=columns)
        return data
    finally:
        cursor.close()
        conn.close()
# 查看DF的某列是否有重复值
def check_duplicate(df,column_name):
    return df[column_name].duplicated().sum()

# 去除DF中某列的重复值(只保留第一个)
def remove_duplicate(df,column_name):
    return df.drop_duplicates(subset=[column_name],keep='first')


# 将DataFrame写入MDB数据库
def write_to_mdb(conn_str, df, table_name):
    """
    将DataFrame写入MDB数据库
    
    参数:
    conn_str: 数据库连接字符串
    df: 要写入的DataFrame
    table_name: 要创建的表名
    """
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    try:
        # 创建表 - 全部使用TEXT类型以避免数值越界
        columns = []
        for col in df.columns:
            columns.append(f"[{col}] TEXT(255)")
        
        create_table = f"CREATE TABLE {table_name} ({', '.join(columns)})"
        cursor.execute(create_table)
        
        # 预处理数据 - 将所有数据转换为字符串
        for _, row in df.iterrows():
            values = []
            for v in row:
                if pd.isna(v):
                    values.append(None)
                else:
                    # 将所有非空值转换为字符串，并限制长度
                    values.append(str(v)[:255])
                    
            placeholders = ','.join(['?' for _ in values])
            insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(insert_query, values)
            
        conn.commit()
        print(f"成功将数据写入表 {table_name}")
    except Exception as e:
        print(f"写入数据时发生错误: {str(e)}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()
# 替换MDB数据库中的表
def replace_table_in_mdb(conn_str, df, table_name):
    """
    替换MDB数据库中的现有表
    参数:
    conn_str: 数据库连接字符串
    df: 要写入的DataFrame
    table_name: 要替换的表名
    """
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    try:
        # 先尝试删除已存在的表
        try:
            cursor.execute(f"DROP TABLE {table_name}")
            conn.commit()
            print(f"已删除原有表 {table_name}")
        except:
            print(f"表 {table_name} 不存在，将创建新表")
        
        # 创建新表
        columns = []
        for col, dtype in df.dtypes.items():
            if 'int' in str(dtype):
                col_type = 'LONG'
            elif 'float' in str(dtype):
                col_type = 'DOUBLE'
            elif 'datetime' in str(dtype):
                col_type = 'DATETIME'
            else:
                col_type = 'TEXT(255)'
            columns.append(f"[{col}] {col_type}")
        
        create_table = f"CREATE TABLE {table_name} ({', '.join(columns)})"
        cursor.execute(create_table)
        
        # 插入数据
        for _, row in df.iterrows():
            values = [None if pd.isna(v) else v for v in row]
            placeholders = ','.join(['?' for _ in values])
            insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(insert_query, values)
        
        conn.commit()
        print(f"成功替换表 {table_name}")
        
    except Exception as e:
        print(f"替换表时发生错误: {str(e)}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

def table_area_by_grade(df, level_columns='质量等级', area_columns='图斑地类面积'):
    """
    构造耕地等级面积统计表
    
    参数:
    df: DataFrame - 包含耕地质量等级和面积数据的DataFrame
    level_columns: str - 表示质量等级的列名
    area_columns: str - 表示面积的列名(单位应为平方米)
    
    返回:
    DataFrame - 格式化的统计表
    """
    calc_df = df.copy()
    
    # 将面积单位(平方米)换算为亩
    calc_df[area_columns]  = calc_df[area_columns].astype(float)
    calc_df[level_columns] = calc_df[level_columns].astype(int)
    # calc_df[area_columns] = calc_df[area_columns] * 0.0015
    
    # 计算各等级面积
    grade_stats = calc_df.groupby(level_columns)[area_columns].sum().reset_index()
    
    # 计算总面积和比例
    total_area = grade_stats[area_columns].sum()
    grade_stats['比例(%)'] = (grade_stats[area_columns] / total_area * 100).round(2)
    
    # 定义等级顺序
    grade_order = {
        '一等': 1, '二等': 2, '三等': 3, '四等': 4, '五等': 5,
        '六等': 6, '七等': 7, '八等': 8, '九等': 9, '十等': 10
    }
    
    # 按等级排序
    grade_stats['排序'] = grade_stats[level_columns].map(grade_order)
    grade_stats = grade_stats.sort_values('排序').drop('排序', axis=1)
    
    # 重命名列
    grade_stats.columns = ['耕地等级', '面积（亩）', '比例(%)']
    
    # 添加合计行
    total_row = pd.DataFrame({
        '耕地等级': ['合计'],
        '面积（亩）': [total_area],
        '比例(%)': [100.00]
    })
    
    # 合并结果
    result = pd.concat([grade_stats, total_row], ignore_index=True)
    
    return result

In [4]:
# 指定MDB文件路径 - 需要包含文件扩展名 2023年
mdb_file_23 = r"D:\worker\工作\work\三普\数据\清镇\耕地质量等级评价数据（农科院信息所提供）\贵阳市2023年\清镇市工作空间.cws\DataTable\OriginalDB.mdb"  # 添加.mdb扩展名

# 使用ODBC连接字符串
conn_str_23 = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=' + mdb_file_23 + ';'
)
# 指定MDB文件路径 - 需要包含文件扩展名
mdb_file2_23_2 = r"D:\worker\工作\work\三普\数据\清镇\耕地质量等级评价数据（农科院信息所提供）\贵阳市2023年\清镇市工作空间.cws\DataTable\ProgenyDB.mdb"  # 添加.mdb扩展名

# 使用ODBC连接字符串
conn_str_23_2 = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=' + mdb_file2_23_2 + ';')

In [5]:
# 指定MDB文件路径 - 需要包含文件扩展名 2022年
mdb_file_22 = r"D:\worker\工作\work\三普\数据\清镇\耕地质量等级评价数据（农科院信息所提供）\贵阳市2022年\清镇市工作空间22年.cws\datatable\OriginalDB.mdb"  # 添加.mdb扩展名

# 使用ODBC连接字符串
conn_str_22 = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=' + mdb_file_22 + ';'
)

In [6]:
# 指定MDB文件路径 - 需要包含文件扩展名 2021年
mdb_file_21 = r"D:\worker\工作\work\三普\数据\清镇\耕地质量等级评价数据（农科院信息所提供）\贵阳市2021年\清镇市工作空间21年.cws\Datatable\OriginalDB.mdb"  # 添加.mdb扩展名

# 使用ODBC连接字符串
conn_str_21 = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=' + mdb_file_21 + ';'
)

In [7]:
# 单元属性表 21年
unit_df_21 = get_table_data(conn_str_21,'耕地资源管理单元属性数据表')
print(unit_df_21.shape[0]),unit_df_21.columns

37578


(None,
 Index(['OBJECTID', 'bsm', 'dlmc', 'ZLDWDM', 'ZLDWMC', 'GDPDJB', '贵州土种', '乡镇名称',
        '市州名', '灌溉能力', '排水能力', '质地构型', '障碍因素', '农田林网化', '生物多样性', '清洁程度', '海拔',
        '分区', '耕层质地', '平差面积', '县名', '速效钾', '有机质', '有效磷', '有效土层厚', '土壤容重',
        '地形部位', '耕地变更', '总分_2021', 'pH', '等级_2021'],
       dtype='object'))

In [8]:
# 单元属性表 22年
unit_df_22 = get_table_data(conn_str_22,'耕地资源管理单元属性数据表')
print(unit_df_22.shape[0]),unit_df_22.columns


38140


(None,
 Index(['OBJECTID', 'DLMC', 'ZLDWDM', 'ZLDWMC', 'GDPDJB', 'ZZSXMC', 'bsm',
        '贵州土种', '乡镇名称', '市州名', '灌溉能力', '排水能力', '质地构型', '障碍因素', '农田林网化', '生物多样性',
        '清洁程度', '海拔', '分区', '耕层质地', '平差面积', '县名', '速效钾', '有机质', '有效磷', '有效土层厚',
        '土壤容重', '地形部位', '耕地变更', '总分_2022', 'pH', '等级_2022', 'Shape_Length',
        'Shape_Area'],
       dtype='object'))

In [9]:
# 单元属性表 23年
unit_df_23 = get_table_data(conn_str_23,'耕地资源管理单元属性数据表')
print(unit_df_23.shape[0])
# 结果数据表 23年
result_df_23 = get_table_data(conn_str_23_2,'耕地质量评价结果数据表')
print(result_df_23.shape[0])
# 连接两个表基于'标识码'
merged_df_23 = pd.merge(unit_df_23, result_df_23, on='标识码', how='left')
print(merged_df_23.shape[0])
merged_df_23.columns


48807
48807
48807


Index(['标识码', '地类名称', '坐落单位代码', '市州名', '县名_x', '乡镇名', '村名', '图斑面积', '图斑地类面积',
       '耕地坡度级别', '土种', '灌溉能力', '排水能力', '质地构型', '障碍因素', '生物多样性', '清洁程度', '海拔',
       '二级农业区', '耕层质地', '平差面积', '有效土层厚', '地形部位', '农田林网化', 'pH', '土壤容重', '速效钾',
       '有机质', '有效磷', '耕地质量变动区', '县名_y', 'F地形部位', 'F灌溉能力', 'F有效土层厚', 'F有机质',
       'F耕层质地', 'F海拔', 'F质地构型', 'F排水能力', 'FPH', 'F速效钾', 'F有效磷', 'F土壤容重',
       'F障碍因素', 'F生物多样性', 'F农田林网化', 'F清洁程度', '评价得分', '质量等级'],
      dtype='object')

In [10]:
# 21年result
result_df_21 = table_area_by_grade(unit_df_21,level_columns='等级_2021',area_columns='平差面积')
result_df_21['面积（亩）'] = result_df_21['面积（亩）']*15
result_df_21

,耕地等级,面积（亩）,比例(%)
0,1,8221.263513,1.40
1,2,12812.366879,2.17
2,3,52673.824420,8.94
3,4,144183.375245,24.47
4,5,149780.460871,25.42
5,6,94640.003555,16.06
6,7,66572.920864,11.30
7,8,40923.911334,6.94
8,9,12720.921466,2.16
9,10,6795.301851,1.15


In [11]:
# 22年result
result_df_22 = table_area_by_grade(unit_df_22,level_columns='等级_2022',area_columns='平差面积')
result_df_22['面积（亩）'] = result_df_22['面积（亩）']*15
result_df_22

,耕地等级,面积（亩）,比例(%)
0,1,8302.820071,1.42
1,2,12819.953606,2.19
2,3,52560.949214,8.99
3,4,144806.555669,24.76
4,5,149855.333693,25.62
5,6,91695.777843,15.68
6,7,66039.870732,11.29
7,8,39877.576657,6.82
8,9,12227.549601,2.09
9,10,6658.662921,1.14


In [12]:
# 23年result
result_df_23 = table_area_by_grade(merged_df_23,level_columns='质量等级',area_columns='图斑地类面积')
result_df_23['面积（亩）'] = result_df_23['面积（亩）']*0.0015
result_df_23

,耕地等级,面积（亩）,比例(%)
0,1,8033.275125,1.36
1,2,13755.402385,2.33
2,3,53933.870238,9.12
3,4,145427.078138,24.59
4,5,152181.484631,25.73
5,6,92828.998346,15.70
6,7,66732.403772,11.28
7,8,40113.820948,6.78
8,9,11927.919610,2.02
9,10,6472.411665,1.09


In [13]:
# 2024年
merged_df_24 = pd.read_excel(r"G:\soil_property_result\qzs\grade_evaluation\result\grade_evaluation_result_have_channeng.xlsx")
# 筛选XZQMC不为空
merged_df_24 = merged_df_24[merged_df_24['XZQMC'].notna()]

In [17]:
map_dict = {"一等":1, "二等":2, "三等":3, "四等":4, "五等":5, "六等":6, "七等":7, "八等":8, "九等":9, "十等":10}
merged_df_24['质量等级'] = merged_df_24['综合质量等级'].map(map_dict)
# 24年result
result_df_24 = table_area_by_grade(merged_df_24,level_columns='质量等级',area_columns='平差面积')
# 将等级替换为数字
result_df_24['面积（亩）'] = result_df_24['面积（亩）']*0.0015
result_df_24

,耕地等级,面积（亩）,比例(%)
0,1,673.989891,0.11
1,2,9214.446425,1.53
2,3,44597.097717,7.41
3,4,70132.145129,11.65
4,5,175750.266222,29.21
5,6,182624.331467,30.35
6,7,70842.272122,11.77
7,8,42397.243476,7.05
8,9,5410.249543,0.90
9,10,103.114446,0.02


In [18]:
# 将所有表写入Excel的不同sheet
tables = {
    '2021年': result_df_21,
    '2022年': result_df_22,
    '2023年': result_df_23,
    '2024年': result_df_24,
}

save_path = r"G:\soil_property_result\qzs\grade_evaluation\result\report"
os.makedirs(save_path, exist_ok=True)

# 使用with语句自动管理资源关闭
with pd.ExcelWriter(os.path.join(save_path, '历史耕地质量等级统计表.xlsx')) as excel_writer:

    for table_name, table_data in tables.items():
        try:
            table_data.to_excel(excel_writer, sheet_name=table_name, index=False)
        except Exception as e:
            print(f"写入表格{table_name}时出现错误: {e}")
            continue